In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/monthly_exercise   

/content/drive/MyDrive/monthly_exercise


### 라이브러리

In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout, Flatten, BatchNormalization, Input, Convolution2D, Activation,TimeDistributed
from keras.layers import Input, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import GlobalAveragePooling1D, Permute, Dropout
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob


train=pd.read_csv('train_features.csv')
train_labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

###각 칼럼 표준화

In [4]:
column_list = ['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']
for column in column_list:
  train[column] = (train[column]-train[column].mean())/train[column].std()

제출 횟수가 제한되어 있어 validation_set 분리  
혼자 테스트할 때는 test_size=0.2, 제출할 때는 0.01로 했습니다.  

In [5]:
train_id, test_id = train_test_split(train['id'].unique(), test_size=0.01, random_state = 2)

LSTM 모델에 넣기 위해서 transpose 해서 (600,6)으로 만들었습니다.

In [6]:
x_val = []
          
for uid in tqdm(test_id):
    temp = np.array(train[train['id'] == uid].iloc[:,2:], np.float32).T
    x_val.append(temp)

x_val = np.array(x_val, np.float32)
x_val = x_val[:,:,:,np.newaxis]
x_val_trans = np.transpose(x_val,(0,2,1,3))
x_val_trans.shape

100%|██████████| 32/32 [00:00<00:00, 485.77it/s]


(32, 600, 6, 1)

In [7]:
x_train = []

for uid in tqdm(train_id):
    temp = np.array(train[train['id'] == uid].iloc[:,2:], np.float32).T
    x_train.append(temp)

x_train = np.array(x_train, np.float32)
x_train = x_train[:,:,:,np.newaxis]
x_train.shape

100%|██████████| 3093/3093 [00:05<00:00, 525.03it/s]


(3093, 6, 600, 1)

In [8]:
train_arg = []
def aug(data, uid, shift = 0):
    shift_data = np.roll(data, shift, axis=2)
    train_arg.append(shift_data)
    
for _ in range(6):
    aug(x_train, 0, _*100)   

In [9]:
for i in range(5):
  train_arg[i] = np.transpose(train_arg[i],(0,2,1,3))

In [10]:
y = train_labels['label']
y_train = y[train_id]
y_test = y[test_id]

y_train = tf.keras.utils.to_categorical(y_train) 
y_test = tf.keras.utils.to_categorical(y_test)

In [11]:
x_arg = np.concatenate([train_arg[0] ,train_arg[1],train_arg[2] ,train_arg[3],train_arg[4]])
y_arg = np.concatenate([y_train,y_train,y_train, y_train, y_train])
x_arg = x_arg.reshape(-1,600,6)
x_arg.shape

(15465, 600, 6)

##    MODEL

In [12]:
def generate_model():
    ip = Input(shape=(600, 6))

    x = Masking()(ip)
    x = LSTM(units = 50, return_sequences = True)(x)
    x = LSTM(units=50)(x)
    x = Dropout(0.3)(x)

  
    y = Reshape((3600,1))(ip)
    y = Conv1D (kernel_size=3, filters=128, strides=3, padding='valid',kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    
    y = Conv1D (kernel_size=3, filters=128, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y)
   
    y = Conv1D (kernel_size=3, filters=128, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y)
    
    y = Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y) 

  
    y = Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y) 
     
    y = Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y) 
     
    y = Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y) 
   
    y = Dropout(0.5)(y)
    y = Flatten()(y)
    
    x = concatenate([x, y])

    out = Dense(61, activation='softmax')(x)

    model = Model(ip, out)
    #model.summary()
    return model
model = generate_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 600, 6)]     0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 3600, 1)      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 1200, 128)    512         reshape[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1200, 128)    512         conv1d[0][0]                     
______________________________________________________________________________________________

제출할 때는 30개 앙상블도 해봤습니다. 숫자가 높아질수록 성능이 좋아졌는데    

private 점수보니 public 데이터에서만 성능 향상이 있었던 것 같습니다.
  
15개 학습을 기록하면 너무 길어지기만 할 것 같아서 1개 돌린 것만 올리겠습니다.

코드는 그대로 실행하시면 됩니다.

In [13]:
num_models=5
model_list=[]

for i in tqdm(range(num_models)):
    model = generate_model()
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    model.fit(x_arg,y_arg, epochs=25, batch_size=32,validation_split=0.2)
    model_list.append(model)
    model.save(f"model_{i}.h5")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/25
387/387 [==============================] - 37s 77ms/step - loss: 2.1647 - accuracy: 0.5282 - val_loss: 1.0447 - val_accuracy: 0.7026
Epoch 2/25
387/387 [==============================] - 28s 71ms/step - loss: 1.0073 - accuracy: 0.7096 - val_loss: 0.7387 - val_accuracy: 0.7947
Epoch 3/25
387/387 [==============================] - 28s 71ms/step - loss: 0.7207 - accuracy: 0.7806 - val_loss: 0.6182 - val_accuracy: 0.8138
Epoch 4/25
387/387 [==============================] - 28s 71ms/step - loss: 0.5660 - accuracy: 0.8172 - val_loss: 0.6030 - val_accuracy: 0.8160
Epoch 5/25
387/387 [==============================] - 28s 71ms/step - loss: 0.4801 - accuracy: 0.8403 - val_loss: 0.5367 - val_accuracy: 0.8477
Epoch 6/25
387/387 [==============================] - 28s 71ms/step - loss: 0.3948 - accuracy: 0.8682 - val_loss: 0.4714 - val_accuracy: 0.8597
Epoch 7/25
387/387 [==============================] - 28s 71ms/step - loss: 0.3531 - accuracy: 0.8821 - val_loss: 0.3795 - val_accuracy:

 20%|██        | 1/5 [11:43<46:53, 703.34s/it]

Epoch 1/25
387/387 [==============================] - 36s 76ms/step - loss: 2.2636 - accuracy: 0.5162 - val_loss: 0.9733 - val_accuracy: 0.7213
Epoch 2/25
387/387 [==============================] - 28s 72ms/step - loss: 0.9656 - accuracy: 0.7280 - val_loss: 0.7019 - val_accuracy: 0.7918
Epoch 3/25
228/387 [================>.............] - ETA: 10s - loss: 0.7365 - accuracy: 0.7698

KeyboardInterrupt: ignored

In [14]:
models = []
for i in tqdm(range(0, 15)):
    model_name = f"model_{i}.h5"
    models.append(keras.models.load_model(model_name))

print(f"{len(models)} models reloaded")


100%|██████████| 15/15 [00:28<00:00,  1.89s/it]

15 models reloaded


In [15]:
x_val_trans = x_val_trans.reshape(-1,600,6)
preds = np.zeros(shape=y_test.shape)
test_preds_list=[]

for model, i in zip(models, range(len(models))):
  b = model.predict(x_val_trans)
  print(f"Model {i+1}")
  preds = preds + b
  test_preds_list.append(b)
    
preds = preds / len(models)


Model 1
Model 2
Model 3
Model 4
Model 5
Model 6
Model 7
Model 8
Model 9
Model 10
Model 11
Model 12
Model 13
Model 14
Model 15


## 제출하기 전에 학습되지 않은 데이터를 이용하여 평가

앙상블된 모델의 성능

In [16]:
def logloss(y_true, y_pred, eps=1e-15):
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -(y_true * np.log(y_pred)).sum(axis=1).mean()

logloss(y_test, preds)

0.5001383715236603

단일 모델 각각의 성능

In [17]:
for j in range(len(test_preds_list)):
  print('model {}'.format(j))
  print(logloss(y_test, test_preds_list[j]))
  # print('\n')

model 0
0.84388995
model 1
1.0600046
model 2
0.88112235
model 3
0.8864809
model 4
0.6570991
model 5
1.1382694
model 6
0.7960012
model 7
0.8260783
model 8
1.0290191
model 9
0.5860408
model 10
0.90896964
model 11
0.57936156
model 12
0.7471938
model 13
0.65527004
model 14
0.6530584


In [18]:
test=pd.read_csv('test_features.csv')

column_list = ['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']
for column in column_list:
  test[column] = (test[column]-test[column].mean())/test[column].std()

x_sub = []

for uid in tqdm(test['id'].unique()):
    temp = np.array(test[test['id'] == uid].iloc[:,2:], np.float32).T
    x_sub.append(temp)

test_X = np.array(x_sub, np.float32)
test_X = test_X[:,:,:,np.newaxis]
test_X.shape


100%|██████████| 782/782 [00:00<00:00, 868.69it/s]


(782, 6, 600, 1)

In [19]:
test_X_trans = np.transpose(test_X,(0,2,1,3))

In [20]:
preds = np.zeros(shape=(782,61))
test_preds_list=[]
for model, i in zip(models, range(len(models))):
    b = model.predict(test_X_trans)
    print(f"Model {i+1}")
    preds = preds + b
    test_preds_list.append(b)
  
preds = preds / len(models)

Model 1
Model 2
Model 3
Model 4
Model 5
Model 6
Model 7
Model 8
Model 9
Model 10
Model 11
Model 12
Model 13
Model 14
Model 15


In [21]:
submission.iloc[:,1:]=preds
submission.to_csv('final2_submission.csv', index=False)